# Подсчет числа пользовательских сессий
Вам необходимо подсчитать число пользовательских сессий в разбивке по доменам на данных из лог-файла.

**Пользовательская сессия** - это пребывание пользователя на сайте такое, что между двумя последовательными кликами проходит не более 30 минут.
Лог-файл такой же, как и на лекции. Находится в HDFS по пути `/lectures/lecture02/data/logsM.txt`

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).appName("Spark SQL seminar").getOrCreate()

## Задание №1
Создайте `DataFrame` из лог-файла. Схему можно скопировать из лекции.

In [ ]:
# Ваш код здесь

## Задание №2
Лог не содержит столбца с доменом. Конечно можно извлечь домен с помощью функции [regexp_extract](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_extract), но мы так делать не будем. Напишите `pandas_udf`, которая будет извлекать домены из столбца `url`. Результаты применения функции поместите в столбец `domain`.

Для извлечения домена можно воспользоваться функцией [urlparse](https://docs.python.org/3/library/urllib.parse.html#urllib.parse.urlparse)

In [ ]:
# Ваш код здесь

## Задание №3
Для разминки давайте подсчитаем сколько дней прошло между первым и последним посещением пользователем нашего домена. Будем считать, что интересующий нас домен `news.mail.ru`. В качестве "уникального" идентификатора пользователя договоримся использовать ip-адрес. Использовать оконные функции в данном задании не надо!

### Задание №3.1
Для выполнения задания №3 понадобится делать операции с датами. Заметьте, что в столбце `timestamp` хранится не настоящий timestamp, а число с датой в формате "yyyyMMddHHmmss". Используя функции из `pyspark.sql.functions`, создайте новый столбец `timestamp`, содержащий в себе UNIX timestamp.

При возникновении ошибок, обратите внимание на типы данных. Возможно их нужно привести.

In [ ]:
# Ваш код здесь

### Задание №3.2
Приведя timestamp к правильному формату, решите исходную задачу. В результате должен получится `DataFrame` с двумя столбцами `ip` и `days`. Отсортируйте результат по столбцу `days` в порядке убывания и выведите первые 20 строк.

In [ ]:
# Ваш код здесь

## Задание №4
Подсчитайте число сессий, которое каждый пользователь (уникальный ip) сделал на домене `news.mail.ru`. Для решения этой задачи потребуется использование оконных функций (что это такое чуть ниже). Для работы с окнами в Spark SQL используется метод [over()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.over). Само окно определяется с помощью класса [Window](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Window). Резудьтатом будет `DataFrame` со столбцами `ip` и `sessions`, отсортированный в порядке убывания числа сессий.

_Оконная функция_ выполняет вычисления для набора строк, некоторым образом связанных с текущей строкой. Можно сравнить её с агрегатной функцией, но, в отличие от обычной агрегатной функции, при использовании оконной функции несколько строк не группируются в одну, а продолжают существовать отдельно. Внутри же, оконная функция, как и агрегатная, может обращаться не только к текущей строке результата запроса.

![](https://www.sqlitetutorial.net/wp-content/uploads/2018/11/SQLite-window-function-vs-aggregate-function.png)

Вот пример, показывающий, как сравнить зарплату каждого сотрудника со средней зарплатой его отдела:

```sql
SELECT depname, empno, salary, avg(salary) OVER (PARTITION BY depname)
  FROM empsalary;
```

```
  depname  | empno | salary |          avg          
-----------+-------+--------+-----------------------
 develop   |    11 |   5200 | 5020.0000000000000000
 develop   |     7 |   4200 | 5020.0000000000000000
 develop   |     9 |   4500 | 5020.0000000000000000
 develop   |     8 |   6000 | 5020.0000000000000000
 develop   |    10 |   5200 | 5020.0000000000000000
 personnel |     5 |   3500 | 3700.0000000000000000
 personnel |     2 |   3900 | 3700.0000000000000000
 sales     |     3 |   4800 | 4866.6666666666666667
 sales     |     1 |   5000 | 4866.6666666666666667
 sales     |     4 |   4800 | 4866.6666666666666667
(10 rows)
```

[Документация PostgreSQL](https://postgrespro.ru/docs/postgrespro/12/tutorial-window)

In [ ]:
# Ваш код здесь

## Задание №5
Нарисуйте гистограмму распределения числа сессий

In [ ]:
# Ваш код здесь

In [ ]:
spark.stop()